In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json

In [9]:
import re

def get_title(row):
    title = row['title']
    id = row['id']
    string = re.sub('[^a-z0-9 ]+', '', title.lower())
    string = re.sub(r'[ ]+', ' ', string).strip().replace(' ', '-')
    return f'{string}-{id}'

In [10]:
from glob import glob

files = glob('url/*.json')
urls = []
for f in files:
    with open(f) as fopen:
        data = json.load(fopen)
        urls_ = ['https://www.wapcar.my/news/' + get_title(d) for d in data['data']['records']]
        urls.extend(urls_)
#         urls.extend(json.load(fopen))
urls = sorted(list(set(urls)))
len(urls)

11315

In [11]:
!mkdir page

In [14]:
urls[0]

'https://www.wapcar.my/news/007s-family-car-the-aston-martin-dbx-is-the-perfect-performance-suv-34135'

In [12]:
r = requests.get(urls[0])
soup = BeautifulSoup(r.content, "lxml")

In [15]:
soup.find('h1', {'class': 'new-info-title'}).text

"007's family car, the Aston Martin DBX is the perfect performance SUV"

In [17]:
soup.find('article', {'class': 'article-content'}).text

"\nSo you're moving to the next stage in life, with two kids and spouse in tow. Mid-life crisis or not, you want a weekend toy, but deep down you know that a 2-door supercar is now out of the question. Yet, you long for something that enthralls, excites and quickens the pulse.\nSomething that is suave like agent 007, practical enough to face the daily grind, with performance that will leave your mouth dry. Enter James Bond's carriage of choice, the Aston Martin DBX, a five-door SUV that effortlessly blends pace, style and usability in one car. Here's why its such a brilliant package.\xa0\n108 years of rich Aston Martin heritage\n"

In [18]:
for no, url in tqdm(enumerate(urls)):
    filename = os.path.join('page', f'{no}.json')
    if os.path.exists(filename):
        continue
    
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)
            
    soup = BeautifulSoup(r.content, "lxml")

    try:
        title = soup.find('h1', {'class': 'new-info-title'}).text
    except Exception as e:
        title = None

    try:
        body = soup.find('article', {'class': 'article-content'}).text
    except Exception as e:
        body = None

    data = {
        'url': url,
        'title': title,
        'body': body,
    }
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)

11315it [29:33,  6.38it/s]
